# Tutorial

## Prepare Inputs

In [3]:
import pandas as pd

### BCRs input file, BCR_original_emb file, and rna input file
1. Ensure that each file includes an index column labeled "barcode". This column serves as a unique identifier for each cell.
2. Verify that the cells are aligned in the same order across all three files.
3. The BCR file must include the following columns: "cdr1", "cdr2", "fwr2", "cdr3", "fwr3", and "fwr4".

In [ ]:
bcr = pd.read_csv("exampledata/example_bcr.csv", index_col="barcode")
rna = pd.read_csv("exampledata/example_rna.csv", index_col="barcode")
assert(bcr.index.tolist() == rna.index.tolist())

In [5]:
bcr.head()

,contig_id,is_cell,high_confidence,length,chain,v_gene,d_gene,j_gene,c_gene,full_length,productive,fwr1,fwr1_nt,cdr1,cdr1_nt,fwr2,fwr2_nt,cdr2,cdr2_nt,fwr3,fwr3_nt,cdr3,cdr3_nt,fwr4,fwr4_nt,reads,umis,raw_clonotype_id,raw_consensus_id,exact_subclonotype_id,sample,label,whole_seq,new_whole_seq,v_identity,SHM
barcode,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
AAACCTGCAATAGCAA-1_06,AAACCTGCAATAGCAA-1_contig_1_06,True,True,619,IGH,IGHV2-5,NaN,IGHJ4,IGHG1,True,True,QITLKESGPTLVKPTQTLTLTCTFS,CAGATCACCTTGAAGGAGTCTGGTCCAACGCTGGTGAAACCCACAC...,GFSLTSGMG,GGGTTCTCACTCACTAGTGGAATGGGT,VGWIRQPPGKALEWLAIIY,GTGGGCTGGATCCGTCAGCCCCCAGGAAAGGCCCTGGAGTGGCTTG...,WDDDD,TGGGATGATGATGAC,RYSPSLKNRLTITKDTSKNQVVLTMTNMDPVDTATYY,CGCTATAGCCCATCTCTGAAGAACAGGCTCACCATCACCAAGGACA...,CALHTITTIFDSW,TGTGCCCTCCATACAATAACTACGATCTTTGACTCCTGG,GQGTLVTVSS,GGCCAGGGAACCCTGGTCACCGTCTCCTCAG,749,5,clonotype455,clonotype455_consensus_1,1,6,spikepositive,QITLKESGPTLVKPTQTLTLTCTFSGFSLTSGMGVGWIRQPPGKAL...,06:QITLKESGPTLVKPTQTLTLTCTFSGFSLTSGMGVGWIRQPPG...,0.03754,0.061453
AAACCTGCACAACTGT-1_06,AAACCTGCACAACTGT-1_contig_2_06,True,True,537,IGH,IGHV1-24,NaN,IGHJ4,IGHM,True,True,QVQLVQSGAEVKKPGASVKVSCKVS,CAGGTCCAGCTGGTACAGTCTGGGGCTGAGGTGAAGAAGCCTGGGG...,GYTLTEL,GGATACACCCTCACTGAATTA,SMHWVRQAPGKGLEWMGGF,TCCATGCACTGGGTGCGACAGGCTCCTGGAAAAGGGCTTGAGTGGA...,DPEDGE,GATCCTGAAGATGGTGAA,TIYAQKFQGRVTMTEDTSTDTAYMELSSLRSEDTAVYY,ACAATCTACGCACAGAAGTTCCAGGGCAGAGTCACCATGACCGAGG...,CATGSGYSGYW,TGTGCAACAGGAAGTGGATATAGTGGCTACTGG,GQGTLVTVSS,GGCCAGGGAACCCTGGTCACCGTCTCCTCAG,16512,52,clonotype278,clonotype278_consensus_1,1,6,spikepositive,QVQLVQSGAEVKKPGASVKVSCKVSGYTLTELSMHWVRQAPGKGLE...,06:QVQLVQSGAEVKKPGASVKVSCKVSGYTLTELSMHWVRQAPGK...,0.00000,0.008596
AAACCTGCAGCCTGTG-1_06,AAACCTGCAGCCTGTG-1_contig_1_06,True,True,656,IGH,IGHV4-59,IGHD3-22,IGHJ4,IGHG1,True,True,QVQLQESGPGLVKPSETLSLICTVS,CAGGTGCAGCTGCAGGAGTCGGGCCCAGGACTGGTGAAGCCTTCGG...,GGSISSY,GGTGGCTCCATCAGTAGTTAC,YWSWIRQPAGKGLEWIGRF,TACTGGAGCTGGATCCGTCAGCCCGCCGGGAAGGGACTGGAGTGGA...,YTSGI,TATACCAGTGGGATC,TNYNPSLKSRVTMSVDTSKNQFSLKLTSVTAADTAVYY,ACCAACTACAACCCCTCCCTCAAGAGTCGAGTCACCATGTCAGTAG...,CARGHPGNYDSSAYYLPFDYW,TGTGCGAGAGGCCACCCCGGGAACTATGATAGTAGTGCTTATTACC...,GQGTLVTVSS,GGCCAGGGAACCCTGGTCACCGTCTCCTCAG,1396,8,clonotype1394,clonotype1394_consensus_1,1,6,spikepositive,QVQLQESGPGLVKPSETLSLICTVSGGSISSYYWSWIRQPAGKGLE...,06:QVQLQESGPGLVKPSETLSLICTVSGGSISSYYWSWIRQPAGK...,0.02397,0.066489
AAACCTGCAGTCAGCC-1_06,AAACCTGCAGTCAGCC-1_contig_2_06,True,True,670,IGH,IGHV1-2,NaN,IGHJ5,IGHG1,True,True,QVQLVQSGAEVKNIGVSVKVSCKAS,CAGGTGCAGCTGGTGCAGTCTGGGGCTGAGGTGAAGAACATTGGGG...,GYTFTDY,GGATACACGTTTACAGACTAC,YIHWVRQAPGQGLEWMGWI,TATATACATTGGGTGCGACAGGCCCCTGGACAAGGGCTTGAGTGGA...,NPNSGG,AACCCAAACAGTGGTGGC,TNYAQKFQGRVTMTRDTSISTAYMELSRLRSDDTAVYY,ACAAATTACGCACAGAAGTTTCAGGGCAGGGTCACCATGACCAGGG...,CARERIAAGDRGWFDPW,TGTGCGAGAGAAAGAATAGCGGCAGGTGATAGGGGGTGGTTCGACC...,GRGTLVTVSS,GGCCGGGGAACCCTGGTCACCGTCTCCTCAG,1167,8,clonotype969,clonotype969_consensus_1,1,6,spikepositive,QVQLVQSGAEVKNIGVSVKVSCKASGYTFTDYYIHWVRQAPGQGLE...,06:QVQLVQSGAEVKNIGVSVKVSCKASGYTFTDYYIHWVRQAPGQ...,0.06081,0.095368
AAACCTGGTTCCTCCA-1_06,AAACCTGGTTCCTCCA-1_contig_2_06,True,True,692,IGH,IGHV1-18,IGHD3-22,IGHJ4,IGHG1,True,True,QVQLVQSGDEVKKPGASVKVSCEAS,CAGGTTCAGCTGGTGCAGTCTGGAGATGAGGTGAAGAAGCCTGGGG...,GYTFISY,GGTTACACCTTTATCAGCTAT,GIAWVRQAPGQGLEWMGWI,GGTATCGCCTGGGTGCGACAGGCCCCTGGACAAGGACTTGAGTGGA...,SVYNGN,AGCGTTTACAATGGTAAC,TDYTQKLQGRVTMTMDTSMTTETLTTTVYMELRSLTSEDTAVYY,ACAGACTATACACAAAAGCTTCAGGGCAGAGTCACCATGACCATGG...,CASFGAANYYDSSGYSIAYW,TGTGCGAGTTTCGGGGCGGCCAATTATTATGATAGTAGTGGTTATT...,GQGTLVTVSS,GGCCAGGGAACCCTGGTCACCGTCTCCTCAG,2431,21,clonotype1466,clonotype1466_consensus_1,1,6,spikepositive,QVQLVQSGDEVKKPGASVKVSCEASGYTFISYGIAWVRQAPGQGLE...,06:QVQLVQSGDEVKKPGASVKVSCEASGYTFISYGIAWVRQAPGQ...,0.11576,0.139594


In [9]:
rna.head()

,IGJ,IGLL5,RP11.685N3.1,RP11.731F5.2,CH17.132F21.1,HIST1H4C,UBE2C,DERL3,TUBA1B,MKI67,XBP1,HIST1H1B,MZB1,CDC20,CENPF,RRM2,HIST1H1A,NKG7,HSP90B1,PLK1,SSR4,ASPM,TOP2A,H2AFZ,CCNB1,KPNA2,TUBB4B,CCNB2,HIST1H3B,TYMS,HMGB2,TPX2,HSPA1A,STMN1,TUBB,CCL4,UBE2S,CDK1,KIAA0101,GAPDH,...,RP1.261D10.2,PWAR6,RND2,ATP6V0D1,TSIX,CTH,RP11.829H16.3,NSRP1,HS3ST1,YY1,RAP1A,INVS,PMAIP1,C12orf29,SATB1,MRPL49,ZNF280B,CYB5A,SMIM13,RP5.1092A11.5,RPL37A,NOTCH4,CSRP2BP,MINPP1,ARHGAP22,AGPAT3,CTD.2287O16.5,DLGAP1.AS1,ZNF346,SCN8A,AZI2,HHLA3,BFSP2.AS1,PIK3CA,TARSL2,DIS3,CTD.2256P15.2,ZNF283,FAHD2B,RP11.485G4.2
barcode,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
AAACCTGCAATAGCAA-1_06,0.345539,0.411600,-0.086132,-0.151236,0.034590,0.737681,0.000468,0.122183,1.349319,0.002903,0.035921,0.018952,0.043660,0.018122,0.028404,-0.003002,0.005605,0.020040,-0.073330,0.003218,0.714892,-0.000220,-0.031642,0.810499,0.051781,0.016719,-0.018649,-0.004919,0.046646,0.001487,0.069803,0.001848,-0.048395,0.093688,-0.166407,0.030407,-0.005516,-0.004993,-0.055218,2.717220,...,0.000543,0.007811,0.003051,0.246436,-0.190030,-0.018938,0.002498,0.756050,0.103422,0.004826,1.477532,0.125185,0.996273,1.533783,0.908495,0.023410,0.026612,0.060665,0.045859,0.000148,3.055260,-0.017244,0.025356,-0.020307,0.023149,0.690991,-0.078326,0.010609,0.097421,-0.000090,0.771746,-0.000238,0.000177,0.016492,0.660869,-0.059796,0.012625,0.007151,0.014972,0.026571
AAACCTGCACAACTGT-1_06,0.117444,0.985152,-0.210269,-0.231358,-0.074831,1.304759,-0.003087,0.564784,0.796385,0.000614,-0.023169,0.022755,0.672100,-0.000089,0.024308,-0.009839,0.003628,0.001453,0.331259,0.006888,0.979608,0.014634,0.006626,0.767119,-0.006551,-0.039703,0.477927,-0.004880,0.015497,-0.003947,0.234597,0.075862,1.571279,0.041362,1.742575,-0.012627,0.061119,0.000616,0.003654,3.030539,...,-0.000530,0.019565,-0.000112,0.050467,0.004443,0.049421,0.000026,0.446601,0.121682,1.148364,0.064260,0.043371,0.135553,0.496781,0.076875,0.518622,0.000649,0.070080,-0.019488,0.000000,3.085822,0.019149,0.012201,0.001204,-0.051617,0.029134,-0.074496,-0.002178,-0.004319,0.000163,0.126555,-0.057113,0.000036,0.127062,0.248703,0.093876,0.005728,0.025548,0.009291,-0.002509
AAACCTGCAGCCTGTG-1_06,0.118185,0.304202,1.553921,-0.071504,-0.152440,1.007265,-0.005297,0.122008,1.350721,0.001795,0.065943,-0.081127,0.109634,0.000556,0.021073,-0.001470,0.023594,0.095886,0.026533,0.005346,0.744243,-0.000755,-0.006712,1.805777,-0.018092,0.028895,-0.102510,-0.005725,0.011422,-0.029690,0.862499,-0.002839,1.786828,-0.003554,1.626199,0.065393,-0.081147,-0.004196,-0.013573,2.076763,...,-0.000622,0.016538,0.000178,0.014220,-0.000625,-0.015750,-0.009427,0.713484,0.151735,0.010404,0.768850,-0.025318,-0.103507,-0.027100,0.158645,-0.003006,0.011188,0.091486,-0.021624,0.000380,3.385806,0.008075,-0.024286,0.035763,-0.059381,-0.086796,0.068959,0.054347,-0.047076,0.000102,0.008371,-0.001826,0.000120,-0.070642,-0.085906,0.005892,-0.018601,0.030213,-0.042815,-0.001975
AAACCTGCAGTCAGCC-1_06,0.123230,1.773176,-0.150341,-0.035437,0.003177,0.666422,0.046367,0.030498,1.141658,0.000546,0.018310,0.030937,0.791835,0.004143,0.019782,-0.006186,-0.003995,0.005343,0.166282,0.022486,0.860325,0.000746,0.003116,0.768593,0.051897,-0.046974,0.703796,0.006419,0.000314,-0.001073,0.819045,-0.017502,0.875111,0.047737,0.651682,0.032515,0.109478,-0.002870,0.055678,2.632764,...,0.013171,0.021848,0.001063,-0.189850,0.097308,0.001836,-0.033620,0.569307,0.014809,1.670692,0.720781,0.025477,1.123293,0.099847,0.180940,-0.029691,-0.020987,0.040255,0.024722,0.000225,2.974856,0.060492,-0.044971,0.016569,0.000125,-0.116955,0.035857,0.056368,-0.032830,0.003976,0.069885,-0.079806,0.000041,0.015928,0.099969,0.084648,0.014874,-0.017749,0.041447,0.007477
AAACCTGGTTCCTCCA-1_06,0.162600,0.300888,-0.164492,0.755047,-0.039643,0.022945,0.017033,0.154383,0.842697,0.007529,0.072672,0.075617,-0.013517,-0.000441,0.017133,-0.004000,0.001304,0.055958,1.432915,-0.000546,-0.030369,0.021825,0.672322,0.570097,-0.

### Genreate original BCR embeddings

First, we download the pre-trained BCR encoder


In [ ]:
from CoMBCR.utils import download_BCRencoder

download_BCRencoder()

Then, we generate the original BCR embeddings by default pretrained encoder


In [ ]:
%load_ext autoreload
%autoreload 2
from CoMBCR.runberta import generate_original_bcremb

berta_emb = generate_original_bcremb("exampledata/example_bcr.csv",
                                     outdir = "example_outdir")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


/home/yipingzou2/.local/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


(1000, 1024)


Here we directly used the original BCR embeddings under the exampledata

In [ ]:
bcrori = pd.read_csv("exampledata/example_bcrori.csv", index_col="barcode")
bcrori.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,984,985,986,987,988,989,990,991,992,993,994,995,996,997,998,999,1000,1001,1002,1003,1004,1005,1006,1007,1008,1009,1010,1011,1012,1013,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023
barcode,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
AAACCTGCAATAGCAA-1_06,-0.324237,-0.228240,-0.241070,0.951063,-0.688552,-0.024392,-0.455126,-0.142796,0.071060,0.872651,0.745679,-1.446963,-0.236816,0.790735,0.163910,0.661154,-0.550283,0.235011,0.808025,-0.403467,0.184209,-0.228641,-0.374704,1.183913,-0.293503,0.191221,0.310355,0.324570,0.176543,-0.835486,-0.059501,-0.328855,0.542304,0.525675,-0.089920,0.128836,0.783455,0.203983,0.075203,-0.177153,...,-0.094010,-0.404888,0.376797,-0.087370,-0.948536,0.096584,0.140622,-0.200298,-0.429650,-0.047751,0.045622,-0.018710,-1.323933,0.300778,0.180471,0.022565,0.397402,-0.907574,0.124854,-0.067931,-0.063159,0.315580,0.733691,0.297291,0.192039,-0.673967,0.207694,-0.596004,-0.139316,0.007377,0.433799,-0.089641,0.199088,-0.619232,0.433234,-0.507768,0.094147,0.622088,-0.438514,-0.548302
AAACCTGCACAACTGT-1_06,-0.836004,-0.281383,-1.062309,0.253088,-0.613961,0.173742,-0.924163,0.295760,0.215318,0.514034,-0.440459,-0.968139,-0.361750,0.431290,0.424335,0.476624,-0.359614,-0.120414,0.485411,-0.968542,-0.255496,0.112555,-0.486354,-0.431345,-0.143314,-0.072575,-0.507828,0.339201,0.075004,0.662511,0.442524,0.681891,0.042501,-0.143170,-0.143484,-0.334561,0.260354,0.505795,-0.106635,0.158460,...,0.233648,0.475217,0.414623,-0.417354,-0.607531,-0.617156,0.299052,0.236040,-0.054911,0.668863,0.245940,0.058224,-0.666202,-0.100924,0.509815,0.913478,0.261814,0.132536,-0.459936,-0.340330,0.192032,-0.404580,0.545928,0.259485,-0.355877,-0.537184,0.451728,-0.365223,-0.894617,-0.321779,0.113509,0.117395,-0.115959,-0.211876,0.793687,-0.666870,-0.068236,0.275671,0.094568,-0.396323
AAACCTGCAGCCTGTG-1_06,-0.388252,-0.081444,-0.394865,1.480731,-0.288159,-0.132324,-0.263037,-0.209172,0.423102,0.569923,0.548954,-0.297909,0.153715,0.049280,0.029028,0.185538,-0.202862,0.061464,0.988190,-0.615843,-0.591859,-0.206216,-0.370368,-0.354970,0.143170,0.526909,-0.164294,0.526467,0.030976,0.038967,-0.102997,-0.237515,0.108536,0.673908,0.070362,-0.204745,0.306753,0.245584,-0.691991,0.013020,...,0.364921,0.327766,-0.018406,-0.729517,0.435331,-0.207413,-0.305142,0.075460,-0.365814,0.550948,0.767987,-0.155384,-0.156725,0.206718,0.361170,0.058065,-0.093767,-0.239658,-0.677152,-0.858531,-0.452283,-0.465134,-0.505716,0.654747,-0.140104,-0.167534,0.126172,-0.279263,-0.661146,-0.449368,-0.231537,-0.353619,0.118815,-1.144104,0.398278,0.067934,-0.181059,0.760884,-0.498049,-0.671089
AAACCTGCAGTCAGCC-1_06,-0.912942,0.000863,0.077432,0.949241,0.020408,0.136745,-0.283928,-0.212797,0.105468,0.711840,0.079063,0.329118,0.250465,0.127045,0.020281,-0.209379,-0.157699,-0.468784,0.452860,-0.343004,-0.088827,0.203237,-0.433501,-0.611193,-0.217648,0.505121,-0.123106,0.057282,0.120223,0.164911,0.086675,-0.025233,-0.148937,0.543882,0.423194,0.138843,0.749822,0.373775,-0.180070,-0.473892,...,0.009824,0.754519,-0.068542,-0.505677,0.351864,-0.172389,0.017181,0.462095,-0.601120,0.685411,0.394452,0.106305,-0.112161,0.306426,0.670460,-0.341604,0.487728,-0.440816,-0.597773,-0.685187,-0.254157,0.161530,-0.042097,0.367366,0.112417,-0.764805,-0.266777,-0.304035,-1.085326,-0.214901,-0.587940,0.107624,-0.320966,-1.219826,0.783442,0.099214,0.109777,0.096048,-0.420305,-1.142159
AAACCTGGTTCCTCCA-1_06,-0.079208,-0.455230,-0.978580,1.132275,-0.574163,0.258197,-0.157592,-0.350995,0.127573,0.343525,0.222817,-0.440587,0.230669,0.282749,-0.011571,-0.539485,0.594990,0.171949,0.717562,-1.071786,-0.574408,0.284977,-0.378849,-0.101788,0.247237,0.361512,-0.230955,0.186483,0.161821,-0.132198,-0.354848,-0.327020,-0.131869,0.283363,0.129220,-0.038910,0.269280,0.665796,-0.712387,-0.365113,...,-0.033064,0.827225,0.002474,-0.587405,-0.087099,0.049133,0.04

In [11]:
assert(bcr.index.tolist() == bcrori.index.tolist())

### Run CoMBCR

## steps
CoMBCR contains the parameters as follows:
1. bcrpath: (Required) The path to the BCR sequences file.
2. rnapath:  (Required) The path to the gene expression file.
3. bcroriginal: (Required) The path to bcr original embedding file.
4. outdir:  (Required) The directory where the best checkpoint file and the output embeddings will be stored.
5. checkpoint: default is "best_network.pth". This parameter specifies the name of the file where the best model checkpoint will be saved.
6. lr: default is 1e-5.
7. lam: default is 1e-1, the inner parameter (Parameter alpha in paper).
8. batch_size: default 256.
9. epochs: default 200.
10. patience: default 15, the patience for early stopping.
11. lr_step: default [30,100], These are the milestones for the MultiStepLR setting, which adjusts the learning rate at specified epochs.
12. encoderprofile_in_dim: default 5000. Adjust this parameter if the number of input genes differs from 5000.
13. separatebatch: The default is False. If set to True, BCRs from different samples will be treated as distinct BCRs. Ensure that your BCR input file contains a "sample" column if you choose to enable this option.

The codes below return numpy arrays for BCR embeddings and gex embeddings. Meanwhile, it will output "bcrembedding.csv" and "gexemedding.csv" under the outdir you designated.  
If CUDA raised error, this is due to the former loaded model. please restart the jupyter notebook and directly run the cell below.

In [ ]:
from CoMBCR.CoMBCR import CoMBCR_main

bcremb, gexemb = CoMBCR_main(bcrpath="exampledata/example_bcr.csv", 
            rnapath="exampledata/example_rna.csv", 
            bcroriginal="exampledata/example_bcrori.csv", 
            outdir="example_outdir",
            epochs=1,
            batch_size=32,
            encoderprofile_in_dim=5000)

/home/yipingzou2/.local/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


learning rate is  1e-05


/home/yipingzou2/.local/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Adjusting learning rate of group 0 to 1.0000e-05.
Adjusting learning rate of group 0 to 1.0000e-05.
Epoch:[0/1]	loss:7.07274	loss_cmc:3.472365	loss_p2p:3.687132	loss_b2b:0.002449


In [ ]:
bcremb.shape

(1000, 256)

In [ ]:
gexemb.shape

(1000, 256)

Read the output files "bcrembedding.csv" and "gexembedding.csv" located in the designated output directory. Please note that these CSV files directly store the numpy arrays and, as such, do not include any "barcode" column. When reading these files, ensure that you do not specify any index column.

In [ ]:
bcremb = pd.read_csv("example_outdir/Embeddings/bcrembeddings")
gexemb = pd.read_csv("example_outdir/Embeddings/gexembeddings")

In [ ]:
bcremb.head()

,0,1,2,3,4,5,6,7,8,9,...,246,247,248,249,250,251,252,253,254,255
0,0.015348,-0.087860,-0.013923,0.056350,-0.017100,0.068572,0.005855,-0.050495,-0.005957,0.008949,...,-0.136085,-0.076334,-0.083860,-0.018186,-0.109076,-0.055655,0.039805,-0.069356,-0.029027,-0.031129
1,0.124641,0.031098,0.027163,0.138613,-0.001733,0.038861,0.051714,0.011094,-0.049876,0.009269,...,-0.106010,0.002925,-0.024259,-0.017597,-0.042338,-0.005912,0.011963,-0.059456,0.017165,0.023257
2,0.035745,-0.049108,-0.037657,0.123449,0.000613,0.055640,0.086152,0.093559,0.029612,-0.020621,...,-0.182125,0.023510,-0.041843,-0.049026,-0.042913,0.086104,-0.060631,-0.022182,0.026433,0.007512
3,0.087479,0.025641,-0.032866,0.033859,0.027408,-0.012790,0.118991,0.037054,-0.002722,-0.013472,...,-0.134776,0.024128,0.035573,-0.098892,0.000892,0.075088,0.008346,-0.024887,-0.018854,0.046138
4,0.013984,-0.057683,-0.067793,0.080526,-0.009838,0.049828,0.068106,0.017483,0.001821,0.027285,...,-0.127777,-0.004428,-0.013432,-0.056396,-0.009095,0.052927,-0.116035,0.022259,-0.001118,-0.014250


In [ ]:
gexemb.head()

,0,1,2,3,4,5,6,7,8,9,...,246,247,248,249,250,251,252,253,254,255
0,-0.048509,-0.050780,0.012092,0.054801,-0.077096,-0.071795,0.120492,0.048243,-0.091755,-0.037499,...,0.030512,-0.069922,-0.017588,-0.057541,-0.033826,-0.059658,-0.007020,-0.018299,0.034806,0.077380
1,0.008329,-0.065424,0.061591,-0.017872,-0.059715,-0.040780,0.160052,-0.033229,-0.061964,-0.026953,...,-0.013495,-0.101155,0.022401,-0.036999,-0.039105,-0.100446,-0.006467,-0.010011,0.036884,0.089870
2,-0.035611,-0.070885,0.032202,0.049993,-0.043414,-0.038092,0.121762,0.008126,-0.090482,-0.002106,...,0.000275,-0.075901,0.017932,-0.014056,-0.033765,-0.051641,-0.003662,-0.027082,0.036087,0.060277
3,-0.063172,-0.066526,0.050900,0.066959,-0.041829,-0.060583,0.122334,-0.033201,-0.063927,0.005277,...,-0.014794,-0.117377,0.009713,-0.037630,-0.051727,-0.059814,-0.001520,-0.034314,0.050281,0.058940
4,-0.012943,-0.077301,-0.010864,0.059000,-0.085983,-0.026742,0.133180,0.004001,-0.066243,-0.045583,...,-0.045505,-0.119614,0.013246,-0.047752,-0.039822,-0.060070,0.023413,0.028433,0.026463,0.055106
